In [5]:
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer
from transformers import pipeline


In [6]:
from google.colab import userdata
from huggingface_hub import notebook_login

notebook_login()

In [7]:
# Load a pre-trained sentence transformer model for embedding
embedding_model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

# Example customer feedback (replace with actual feedback from your eCommerce platform)
feedback = [
    "The checkout process is too complicated and confusing.",
    "I had trouble finding product reviews on the mobile app.",
    "The search function does not return relevant results.",
    "Shipping options are too limited, and there should be more affordable choices.",
    "The product pages load too slowly, especially on mobile devices.",
]

# Generate embeddings for the feedback
feedback_embeddings = embedding_model.encode(feedback)

In [16]:
access_token = userdata.get('HF_TOKEN')
# Create a FAISS index
dimension = feedback_embeddings.shape[1]  # The dimension of embeddings
index = faiss.IndexFlatL2(dimension)      # L2 distance for similarity search
index.add(np.array(feedback_embeddings))  # Add feedback embeddings to the index

# Save feedback and index for later use (optional, if needed)
np.save("feedback.npy", feedback)
faiss.write_index(index, "feedback_index.faiss")

# Load the text generation model (e.g., GPT-Neo) for generating suggestions
generator = pipeline("text-generation", model= "meta-llama/Llama-3.2-1B-Instruct", device=-1, token=access_token)  # CPU usage


Device set to use cpu


In [11]:
!pip install huggingface hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 33.2 MB/s eta 0:00:00


In [17]:

# Utility function to retrieve relevant feedback
def retrieve_feedback(query, top_k=3):
    query_embedding = embedding_model.encode([query])
    distances, indices = index.search(np.array(query_embedding), top_k)
    return [feedback[idx] for idx in indices[0]]

# Main function to provide suggestions based on feedback
def suggest_enhancements(query):
    # Step 1: Retrieve relevant feedback based on the query
    retrieved_feedback = retrieve_feedback(query)
    print("Relevant Feedback Retrieved:")
    for fdbk in retrieved_feedback:
        print(f"- {fdbk}")

    # Step 2: Prepare the prompt for the LLM to generate suggestions
    context = " ".join(retrieved_feedback)
    prompt = (
        f"Based on the following customer feedback: {context}\n"
        "Suggest improvements or enhancements for the eCommerce website/app to address these issues."
    )

    # Step 3: Generate actionable suggestions using the LLM
    response = generator(prompt, max_length=500)[0]['generated_text']

    # Print the generated response
    print("\nSuggested Enhancements:")
    print(response)

# Example usage
if __name__ == '__main__':
    # Enter a query to start the RAG system (e.g., "improve checkout experience")
    query = input("Enter the area of improvement you're looking for suggestions on: ")
    suggest_enhancements(query)


Enter the area of improvement you're looking for suggestions on: Checkout and shipping


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Relevant Feedback Retrieved:
- The checkout process is too complicated and confusing.
- Shipping options are too limited, and there should be more affordable choices.
- I had trouble finding product reviews on the mobile app.

Suggested Enhancements:
Based on the following customer feedback: The checkout process is too complicated and confusing. Shipping options are too limited, and there should be more affordable choices. I had trouble finding product reviews on the mobile app.
Suggest improvements or enhancements for the eCommerce website/app to address these issues. 

**Complicated and Confusing Checkout Process**

1. **Simplify the checkout process**: 
	* Remove unnecessary fields and reduce the number of steps.
	* Use clear and concise language in the checkout process.
	* Consider adding a "quick checkout" option for users who are not ready to purchase immediately.
2. **Improve the checkout experience**:
	* Use a more modern and responsive checkout design.
	* Consider adding a "bu

Relevant Feedback Retrieved:
- I had trouble finding product reviews on the mobile app.
- Shipping options are too limited, and there should be more affordable choices.
- The checkout process is too complicated and confusing.

Suggested Enhancements:
Based on the following customer feedback: I had trouble finding product reviews on the mobile app. Shipping options are too limited, and there should be more affordable choices. The checkout process is too complicated and confusing.
Suggest improvements or enhancements for the eCommerce website/app to address these issues. Here are some possible solutions:

**Improvement 1: Simplify product reviews and search functionality**

* Implement a robust search bar that allows customers to find specific products quickly.
* Introduce a filtering system to categorize products by price, brand, and rating, making it easier for customers to find what they're looking for.
* Consider adding a "recommended products" section that suggests similar products based on customer reviews and ratings.

**Improvement 2: Expand shipping options and pricing**

* Offer more affordable shipping options, such as standard or expedited shipping, to cater to a wider range of customers.
* Introduce a price comparison feature that allows customers to compare prices across different sellers and shipping options.
* Consider implementing a "buy now" or "order now" button that automatically applies the chosen shipping option and price.

**Improvement 3: Streamline checkout process**

* Simplify the checkout process by removing unnecessary steps, such as requiring customers to create an account or enter their email address.
* Introduce a "quick checkout" option that allows customers to proceed with their purchase without entering their payment information.
* Consider implementing a "save for later" feature that allows customers to save their cart and pick it up later.

**Additional suggestions**

* Implement a "customer reviews" section that showcases customer feedback and ratings for each product.
* Consider adding a "returns and refunds" section that provides clear instructions and FAQs for customers.
* Introduce a "contact support" feature that allows customers to reach out to the company's customer support team for assistance.

By addressing these issues, the eCommerce website/app can improve its user experience, increase customer satisfaction, and drive more sales.



In [13]:
from huggingface_hub import notebook_login

notebook_login()